#Written on Python ver. 3.9.6

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import time as sleep
# 웹캠에서 비디오 스트림 시작
cap = cv2.VideoCapture(0)
sleep.sleep(1)

# 첫 번째 프레임에서 ROI 지정
ret, frame = cap.read()
r = cv2.selectROI(frame)
track_window = (int(r[0]), int(r[1]), int(r[2]), int(r[3]))

# MeanShift를 사용하여 객체 추적 초기화
roi = frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, (0., 60., 32.), (180., 255., 255.))
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# 추적을 위한 종료 기준 설정
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
    ret, frame = cap.read()
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        
        # MeanShift를 사용하여 다음 위치 계산
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # 바운딩 박스 그리기
        x, y, w, h = track_window
        img2 = cv2.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        cv2.imshow('Tracking', img2)
        
        if cv2.waitKey(1) & 0xFF == 27:  # ESC 키를 누르면 종료
            break

cap.release()
cv2.destroyAllWindows()
